In [1]:
import os
import cv2
import numpy as np
import glob

In [2]:
# image folder 생성
if not os.path.isdir('images'):
    os.mkdir('images')

In [3]:
# src.mp4 동영상으로 부터 이미지 프레임 추출
src_idx = 0# 소스 프레임이 몇 번째인지 인덱스 설정
cap1 = cv2.VideoCapture('../1/src.mp4')
while True:
    retval, frame = cap1.read()
    if not retval:
        break
    file_name = f'./images/src/src_{src_idx:03d}.jpg'# 프레임별로 인덱스를 정하기 위함.
    cv2.imwrite(file_name, frame)
    src_idx += 1

if cap1.isOpened():
    cap1.release()

In [4]:
# dst.mp4 동영상으로 부터 이미지 프레임 추출
dst_idx = 0
cap2 = cv2.VideoCapture('../1/dst.mp4')
while True:
    retval, frame = cap2.read()
    if not retval:
        break
    file_name = f'./images/dst/dst_{dst_idx:03d}.jpg'
    cv2.imwrite(file_name, frame)
    dst_idx += 1

if cap2.isOpened():
    cap2.release()

In [5]:
# homography 계산
pts_src = np.array([[0, 0], [1279, 0], [1279, 719], [0, 719]])# 1 뺀값 (해당 영상은 1280x720 src영상을 dst영상에 붙이기 위함)
pts_dst = np.array([[232, 100],[606, 92],[602, 352],[236, 316]])# dst영상에서 src영상을 붙일 영역
h, status = cv2.findHomography(pts_src, pts_dst)
print(h)

[[ 2.10922464e-01  1.21207918e-02  2.32000000e+02]
 [-1.86268350e-02  3.09197640e-01  1.00000000e+02]
 [-1.34477700e-04  2.77860572e-05  1.00000000e+00]]


In [6]:
# 이미지 합성 함수
def synthesize(im_src, im_dst):
    im_warp = cv2.warpPerspective(im_src, h, (im_dst.shape[1], im_dst.shape[0]))
    cv2.fillConvexPoly(im_dst, pts_dst.astype(int), (0, 0, 0), cv2.LINE_AA)
    return im_warp + im_dst

In [7]:
# 프레임별 이미지 합성
file_num = min(len(glob.glob("./images/src/src*.jpg")), len(glob.glob("./images/dst/dst*.jpg")))
for i in range(file_num):
    im_src = cv2.imread("./images/src/src_" + str(i).zfill(3) + ".jpg")
    im_dst = cv2.imread("./images/dst/dst_" + str(i).zfill(3) + ".jpg")
    im_comp = synthesize(im_src, im_dst)
    
    cv2.imwrite("./images/synth/synth_" + str(i).zfill(3) + ".jpg", im_comp)

In [9]:
# 최종 동영상 생성
file_list = os.listdir("./images/synth")
fps = 20# fps 설정
fcc = cv2.VideoWriter_fourcc('D', 'I', 'V', 'X')# 코덱 설정
width = 854
height = 480

out = cv2.VideoWriter('virtualboard.avi', fcc, fps, (width, height))
for f in file_list:
    fn = f'./images/synth/{f}'
    img = cv2.imread(fn)
    out.write(img)
    
out.release()